# Scenario 3 - Exercise

## create dim table regions

In [ ]:
from google.cloud import bigquery

# TODO : Change to your project id
PROJECT_ID = "sparsh-data-eng-on-gcp"
TARGET_TABLE_ID = "{}.dwh_bikesharing.dim_regions".format(PROJECT_ID)


def create_dim_table(PROJECT_ID, TARGET_TABLE_ID):
    client = bigquery.Client()
    job_config = bigquery.QueryJobConfig(
        destination=TARGET_TABLE_ID,
        write_disposition='WRITE_TRUNCATE')

    sql = """SELECT CAST(region_id AS STRING) as region_id, name
          FROM `{}.raw_bikesharing.regions` regions
          ;""".format(PROJECT_ID)

    query_job = client.query(sql, job_config=job_config)

    try:
        query_job.result()
        print("Query success")
    except Exception as exception:
        print(exception)

In [ ]:
create_dim_table(PROJECT_ID, TARGET_TABLE_ID)

## Create fact table daily by gender region

In [ ]:
import sys
from google.cloud import bigquery

# TODO : Change to your project id
PROJECT_ID = "sparsh-data-eng-on-gcp"
target_table_id = "{PROJECT_ID}.dwh_bikesharing.fact_region_gender_daily".format(
    PROJECT_ID)


def create_fact_table(PROJECT_ID, target_table_id):
    load_date = sys.argv[1]  # date format : yyyy-mm-dd
    print("\nLoad date:", load_date)

    client = bigquery.Client()
    job_config = bigquery.QueryJobConfig(
        destination=target_table_id,
        write_disposition='WRITE_APPEND')

    sql = """SELECT DATE(start_date) as trip_date,
                region_id,
                member_gender,
                COUNT(trip_id) as total_trips
                FROM `{PROJECT_ID}.raw_bikesharing.trips` trips
                JOIN `{PROJECT_ID}.raw_bikesharing.stations` stations
                ON trips.start_station_id = stations.station_id
                WHERE DATE(start_date) = DATE('{load_date}') AND member_gender IS NOT NULL
                GROUP BY trip_date, region_id, member_gender
                ;""".format(PROJECT_ID=PROJECT_ID, load_date=load_date)

    query_job = client.query(sql, job_config=job_config)

    try:
        query_job.result()
        print("Query success")
    except Exception as exception:
        print(exception)

In [ ]:
create_fact_table(PROJECT_ID, target_table_id)

## Answers

In [ ]:
SELECT
name as region_name, total_trips
FROM `sparsh-data-eng-on-gcp.dwh_bikesharing.fact_region_gender_daily`fact
JOIN `sparsh-data-eng-on-gcp.dwh_bikesharing.dim_regions` dim
ON fact.region_id = dim.region_id
WHERE DATE(trip_date) = DATE('2018-01-02')
AND member_gender = 'Female'
ORDER BY total_trips desc
LIMIT 3;